### Earth weather data since 1940 (max allowed) from Open-Meteo's API


For description of  locations considered and overall process, check notebook [earth_weather_data_group_1.ipynb][mylink]

[mylink]: earth_weather_data_group1.ipynb

In [1]:
#____ Library Import ____

#! pip install openmeteo_requests
#! pip install requests_cache
#! pip install retry_requests

import openmeteo_requests         # Open-Meteo weather API
import requests_cache             # chaching responses to reduce repetitive requests
import pandas as pd
from datetime import datetime, timezone
import time
from retry_requests import retry

In [2]:
#____ Caching & Client ____

cache_session = requests_cache.CachedSession(".cache", expire_after = -1)   # caching object; doesn't expire (-1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)     # in case the request fails
openmeteo = openmeteo_requests.Client(session = retry_session)              # Open-Meteo client

In [3]:
#____ Coords to Process ____

coordinates = [
    {"name": "McMurdo", "latitude": -77.8460, "longitude": 166.6720},       # McMurdo Station, Antarctica
    {"name": "Death_Valley", "latitude": 36.2425, "longitude": -116.8258},  # Death Valley, United States 
    {"name": "Zurich", "latitude": 47.38, "longitude": 8.54}               # Zurich, Switzerland
]

In [4]:
#____ Setup ____
url = "https://archive-api.open-meteo.com/v1/archive"       # base URL

In [5]:
now_utc = datetime.now(timezone.utc) 
end_date = now_utc.strftime("%Y-%m-%d")  

In [6]:
# params contains the query details 
base_params = {
    "start_date": "1940-01-01",
    "end_date": "2024-03-16",
    "hourly": ["temperature_2m", "relative_humidity_2m", "rain", "direct_radiation_instant"]
}

In [7]:
#____ Request ____
results = [] 
hourly_dataframe = pd.DataFrame()

for coords in coordinates:
    params = base_params.copy()  # making a copy to avoid modifying the base parameters
    loc_name = coords["name"]
    params["latitude"] = coords["latitude"]
    params["longitude"] = coords["longitude"]


    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]  

    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()}m ASL (*Above Sea Level)")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()}s")

    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_rain = hourly.Variables(2).ValuesAsNumpy()
    hourly_direct_radiation_instant = hourly.Variables(3).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    hourly_data["latitude"] = params["latitude"]
    hourly_data["longitude"] = params["longitude"]
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["rain"] = hourly_rain
    hourly_data["direct_radiation_instant"] = hourly_direct_radiation_instant

    hourly_dataframe = pd.DataFrame(data = hourly_data)


    results.append({
        "coordinates": coords,  
        "weather_data": hourly_dataframe.to_dict("records") 
    })

    temp_df = pd.DataFrame(hourly_data) 


    hourly_dataframe = pd.concat([hourly_dataframe, temp_df], ignore_index=True) 
    hourly_dataframe.to_json(f"../data/weather_data_{loc_name}.json", orient="records")

    time.sleep(60)  


Coordinates -77.85588836669922°N 166.7796630859375°E
Elevation 36.0m ASL (*Above Sea Level)
Timezone None None
Timezone difference to GMT+0 0s
Coordinates 36.239013671875°N -116.80104064941406°E
Elevation -86.0m ASL (*Above Sea Level)
Timezone None None
Timezone difference to GMT+0 0s
Coordinates 47.34621810913086°N 8.54337215423584°E
Elevation 412.0m ASL (*Above Sea Level)
Timezone None None
Timezone difference to GMT+0 0s
